In [1]:
import sys
sys.path.append("../src")
import pandas as pd
import numpy as np
from tqdm import tqdm
import pickle
import lightgbm
import pickle
from dataLoad import testLoad
from metric import ndcg_calculator, hit_at_k

path= "../dataset/"
test, final_sumbission, test_user_label = testLoad(path)

In [2]:
test = test.sort_values(by ="timestamp", ascending = True)\
        .reset_index(drop=True).drop(columns=["timestamp","click_count_normalized","user_click_count_normalized"])

In [2]:
with open("../output/first_batch2000lgbm_model.pkl", 'rb') as f:
    first_batch2000lgbm_model = pickle.load(f)

--- LgbmRanker + ALS + MP + Word2Vec Ensemble      
      
feature :      
*    datset - basic features from featureExtract      
*    Mp - general most popular feature      
*    ALS - user / item factor      
<!-- *    Word2Vec - item segmentation       -->
      
candidate :      
*    Mp  - general/recent weeks Most popular      
<!-- *    Word2Vec - similar items       --> 

In [4]:
# # load features
# basic_7_feature = pd.read_parquet("../output/basic_7_feature.parquet")
general_MP_feature = pd.read_parquet("../output/general_MP_feature.parquet")
user_factor_als = pd.read_parquet("../output/user_factor_als.parquet")
item_factor_als = pd.read_parquet("../output/item_factor_als.parquet")

# # load candidates
# recent30days_MP_candidate = pd.read_parquet("../output/recent30days_MP_candidate.parquet")
# popular_items_cand = pd.read_parquet("../output/popular_items_cand.parquet")

In [5]:
# candidate_1 = recent30days_MP_candidate
# candidate_2 = popular_items_cand
# candidate = [candidate_1, candidate_2]
# cand = pd.concat(candidate)
# cand.drop_duplicates(subset=['user','item_id'],inplace=True)

# print('cand 데이터 수:',len(cand))
# print('cand nunique: ', cand.item_id.nunique())

cand 데이터 수: 119636648
cand nunique:  177


In [5]:
# add feature 
candidate = pd.merge(test, general_MP_feature, how='left', on='item_id')
candidate = pd.merge(candidate, item_factor_als, how="left", on="item_id")
candidate = pd.merge(candidate, user_factor_als, how="left", on="user")

In [ ]:
candidate = candidate.fillna(0).astype(int)  # astype int, fillna

In [ ]:
'''
OOM issue - 대안으로 candidate를 포기하고, feature 만 진행합니다.
'''

In [ ]:
def make_label(train_valid):
    label_df = train_valid[["user","item_id"]]
    label_df.drop_duplicates(subset=["user","item_id"],inplace=True)
    label_df["click_num"] = 1
    return label_df

def label_create(train, label_df):
    train = pd.merge(train, label_df, how="left", on=["user","item_id"])
    train["click_num"] = train["click_num"].fillna(0)
    return train

In [ ]:
label_df = make_label(test_user_label)
candidate_add_features = label_create(candidate, label_df)

In [3]:
'''
OOM issue로 인해 여기까지 완성한 data import 후 다음 단계로 진행합니다
'''
test_candidate_add_features = pd.read_parquet("../output/test_candidate_add_features.parquet")

In [ ]:
# print(candidate_add_features[candidate_add_features["click_num"]==1].count()) # 46771
# print(candidate_add_features[candidate_add_features["click_num"]==0].count()) # 102260423

In [4]:
def lgbm_preprocess(train):
    X_train = train.drop(columns=["click_num"])
    y_train = train["click_num"]
#     group_dict = train.groupby("user")["user"].count().to_dict()
    item_idx = X_train["item_id"].copy()
    user_idx = X_train["user"].copy()
    del X_train["item_id"], X_train["user"]
    return X_train, y_train, item_idx, user_idx

def make_groups(X_train):
    group_dict = []
    batchsize = len(X_train)
    while True:
        if batchsize >= 2000:
            group_dict.append(2000)
            batchsize = batchsize - 2000
        else:
            group_dict.append(batchsize)
            break
            
def train_LgbmRanker_batch(X_train, y_train, model_params, batch_size):
    model = first_batch2000lgbm_model
    num_batches = X_train.shape[0] // batch_size
    for i in tqdm(range(0, X_train.shape[0], batch_size), total=num_batches):
        X_batch = X_train[i:i + batch_size]
        y_batch = y_train[i:i + batch_size]
        group_dict = []
        batchsize = len(X_batch)
        while batchsize !=0 :
            if batchsize >= 1000:
                group_dict.append(1000)
                batchsize = batchsize - 1000
            else:
                group_dict.append(batchsize)
                batchsize -= batchsize 
        model.fit(
        X=X_batch,
        y=y_batch,
        group=group_dict,
        ) 
    feature_importances_df = pd.DataFrame(dict(zip(X_train.columns, model.feature_importances_)), index=["feature_importances"]).T
    return model, feature_importances_df

    
def valid_evaluation(X_train, train_valid, sample_sumbission, model, feature_importances_df, item_idx, user_idx): 
    print(feature_importances_df)
    pred = model.predict(X_train)
    X_train["pred"] = pred
    X_train["item_id"] = item_idx
    X_train["user"] = user_idx
    
    print("performance")
    # each user pred 10 items
    lgbm_sub_df = X_train.sort_values(by="pred", ascending=False).groupby("user").head(10)
    lgbm_user_items_dict = lgbm_sub_df.groupby("user")["item_id"].unique().to_dict()
    sample_sumbission["predicted_list"] = sample_sumbission["user"].apply(lambda x: lgbm_user_items_dict.get(x, []))

    # print("lgbm ndcg:", ndcg_calculator(sample_sumbission, train_valid, 10))
        
    return X_train, sample_sumbission

In [5]:
import gc
gc.collect()

0

In [6]:
model_params={"num_leaves":150 ,"learning_rate":0.005,"n_estimators":35}

In [7]:
X_train, y_train, item_idx, user_idx = lgbm_preprocess(test_candidate_add_features)
model, feature_importances_df = train_LgbmRanker_batch(X_train, y_train, model_params, 10000)

X_train, final_sumbission = \
     valid_evaluation(X_train, test_user_label, final_sumbission, model, feature_importances_df, item_idx, user_idx)

703it [03:55,  2.98it/s]                         


             feature_importances
day_of_week             0.000000
days                    0.000000
hour                   27.127226
weeks                   0.000000
cumcount             1144.737772
...                          ...
user_45                 0.000000
user_46                 0.000000
user_47                 0.000000
user_48                 0.000000
user_49                 0.000000

[109 rows x 1 columns]


C:\Users\wikid\AppData\Local\Temp\ipykernel_15652\310903693.py:48: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_train["pred"] = pred
C:\Users\wikid\AppData\Local\Temp\ipykernel_15652\310903693.py:49: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  X_train["item_id"] = item_idx
C:\Users\wikid\AppData\Local\Temp\ipykernel_15652\310903693.py:50: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once usin

performance


In [40]:
def _ndcg_calculator(gt, rec, idcg):
    if not isinstance(rec, list):
        rec = [rec]
    dcg = 0.0
    if rec in gt:
            dcg = 1.0 / np.log(2)
    return dcg / idcg

def ndcg_calculator(answer, submission, n=10):
    idcg = sum((1.0 / np.log(i + 1) for i in range(1, n + 1)))
    assert (answer.user != submission.user).sum() == 0
    ndcg_list = []
    for (_, row_answer), (_, row_submit) in zip(answer.iterrows(), submission.iterrows()):
        ndcg_list.append(_ndcg_calculator(row_answer.item_id, row_submit.predicted_list, idcg))
    ndcg_score = sum(ndcg_list) / len(answer)
    return ndcg_score

In [ ]:

def _ndcg_calculator(gt, rec, idcg):
    if not isinstance(rec, list):
        rec = [rec]
    dcg = 0.0
    for i, r in enumerate(rec):
        if r in set(gt):
            dcg += 1.0 / np.log(i + 2)
    return dcg / idcg

def ndcg_calculator(answer, submission, n=10):
    idcg = sum((1.0 / np.log(i + 1) for i in range(1, n + 1)))
    assert (answer.user != submission.user).sum() == 0
    ndcg_list = []
    for (_, row_answer), (_, row_submit) in zip(answer.iterrows(), submission.iterrows()):
        ndcg_list.append(_ndcg_calculator(row_answer.item_id, row_submit.predicted_list, idcg))
    ndcg_score = sum(ndcg_list) / len(answer)
    return ndcg_score

In [44]:
final_sumbission

,user,predicted_list
0,0,"[396362, 888904, 228561, 18748, 99247, 784458,..."
1,1,"[148090, 293913, 839005, 293981, 597318, 20430..."
2,2,"[832922, 247703, 267761, 710805, 41459, 716642..."
3,3,"[544492, 614476, 949905, 560874, 126927, 96282..."
4,4,"[641013, 577279, 168299, 486989, 895860, 73144..."
...,...,...
587481,829386,"[813829, 810147, 554650, 130000, 893014]"
587482,829390,"[47147, 945164, 712124, 310178, 866254, 851941..."
587483,829393,"[115233, 492271]"
587484,829397,"[503200, 517483]"


In [46]:
%%time
mf_ndcg = ndcg_calculator(test_user_label, final_sumbission, 10)

print("performance")
print(f"nDCG@10(mf_ALS): {mf_ndcg:.4f}")

performance
nDCG@10(mf_ALS): 0.0160
CPU times: total: 3min 1s
Wall time: 3min 1s


In [42]:
%%time
mf_hit = hit_at_k(test_user_label, final_sumbission, 10)

print("performance")
print(f"hit@10(mf_ALS): {mf_hit:.4f}")

performance
hit@10(mf_ALS): 0.0270
CPU times: total: 1min 30s
Wall time: 1min 31s


In [43]:
%%time
mf_hit = hit_at_k(test_user_label, final_sumbission, 1)

print("performance")
print(f"hit@1(mf_ALS): {mf_hit:.4f}")

performance
hit@1(mf_ALS): 0.0270
CPU times: total: 1min 31s
Wall time: 1min 31s


In [ ]:
# X_train.to_parquet("../output/batch1000_lgbm_X_train.parquet", index=False)
# pickle.dump(model, open("../output/batch1000_lgbm_model.pkl", "wb"))